In [24]:
import numpy as np
import os
import pandas as pd 

def create_cue_dynam(highProb=0.7, lowProb=0.3, neutral=1.0, trials_per_cue=40):
    cue_data = {"cue_names": ["Sea animal",  "Water vessel",  "Neutral"],
                "cue_letters": ["SA", "WV", "X"],
                "cue_color": [[0.4, 0.6, 1.0], [-0.4, -0.2, 0.4], [-1, -1, -1]],
                "cue_highProb_cats": [["dolphin", "whale"], ["speedboat", "submarine"], 
                                    ["dolphin", "whale", "speedboat", "submarine"]],
                
                "cue_lowProb_cats": [["speedboat", "submarine"], ["dolphin", "whale"],
                                    ["dolphin", "whale", "speedboat", "submarine"]]}

    cue_data["cue_highProb"] = []
    cue_data["cue_lowProb"] = []
    
    for cue_id, cue in enumerate(cue_data["cue_names"]):
        if cue != "Neutral":
            cue_data["cue_highProb"].append([np.round(highProb / len(cue_data["cue_highProb_cats"][cue_id]), 2)] * len(cue_data["cue_highProb_cats"][cue_id]))
            cue_data["cue_lowProb"].append([np.round(lowProb / len(cue_data["cue_lowProb_cats"][cue_id]), 2)] * len(cue_data["cue_lowProb_cats"][cue_id]))
        
        else:
            cue_data["cue_highProb"].append([np.round(neutral / len(cue_data["cue_highProb_cats"][cue_id]), 3)] * len(cue_data["cue_highProb_cats"][cue_id]))
            cue_data["cue_lowProb"].append([np.round(neutral / len(cue_data["cue_lowProb_cats"][cue_id]), 3)] * len(cue_data["cue_highProb_cats"][cue_id]))
            
    cue_data["high_prob_trials"] = [np.array(x) * trials_per_cue for x in cue_data["cue_highProb"]] 
    cue_data["low_prob_trials"] =  [np.array(x) * trials_per_cue for x in cue_data["cue_lowProb"]]
    
    return cue_data
    

def build_constrained_order(df, seed=None, max_unexpected_run=1):
    rng = np.random.default_rng(seed)

    remaining = df.copy()
    ordered_rows = []

    last_target = None
    unexpected_run = 0

    while len(remaining) > 0:

        # valid candidates mask
        valid_mask = np.ones(len(remaining), dtype=bool)

        # Rule 1 — no same target twice
        if last_target is not None:
            valid_mask &= (remaining["target"].values != last_target)

        # Rule 2 — max unexpected run
        if unexpected_run >= max_unexpected_run:
            valid_mask &= (remaining["expectation"].values != "unexpected")

        valid = remaining[valid_mask]

        # if dead end → restart whole sequence
        if len(valid) == 0:
            return build_constrained_order(df, seed=rng.integers(0,1e9))

        # pick random valid row
        choice_idx = rng.integers(len(valid))
        row = valid.iloc[choice_idx]

        ordered_rows.append(row)

        # update state
        last_target = row["target"]
        if row["expectation"] == "unexpected":
            unexpected_run += 1
        else:
            unexpected_run = 0

        # remove selected row
        remaining = remaining.drop(valid.index[choice_idx])

    return pd.DataFrame(ordered_rows).reset_index(drop=True)



def create_block_trials(stim_path, cue_data, random_seed, long_isi=0.1): 
    categories = os.listdir(stim_path)
    stimuli = []
    for cat in categories:
        cat_path = os.path.join(stim_path, cat)
        files = os.listdir(cat_path)
        stimuli.extend([f".\\stimuli\\{cat}\\{x}" for x in files])

    stimuli = np.array(stimuli)[np.argsort(stimuli)]
    data = {"target_id": [],
            "distractor_id": [],
            "target": [],
            "distractor": [],
            "expectation": [],
            "mask_ISI": [],
            "cue": [],
            "cue_color": [],
            "cue_letter": [],
            "target_name": [],
            "target_cat": [],}

    mask_type = [0.017, long_isi]
    distractors = stimuli[np.char.count(stimuli, "mask") > 0]
    distractor_ids = np.arange(len(stimuli))[np.isin(stimuli, distractors)]
    local_distractor_ids = np.arange(0, len(distractors))

    for cue_id, cue in enumerate(cue_data["cue_names"]):
        high_cats = np.array(cue_data["cue_highProb_cats"][cue_id])
        low_cats = np.array(cue_data["cue_lowProb_cats"][cue_id])
        
        if cue != "Neutral":
            for i, l_cat in enumerate(low_cats):
                l_cat_stim = stimuli[np.char.count(stimuli, l_cat) > 0]
                targets = l_cat_stim[np.char.count(l_cat_stim, "mask") == 0]
                target_ids = np.arange(len(stimuli))[np.isin(stimuli, targets)]
            

                trial_collectos = []
                for mask in mask_type:
                
                    h_trials = cue_data["low_prob_trials"][cue_id][i]
                    split_h_trials = int(h_trials // 2)
                    data["target_id"].extend(np.repeat(target_ids , split_h_trials))
                    data["target"].extend(np.repeat(targets, split_h_trials))
                    
                    target_names =[x.split("\\")[-1] for x in targets]
                    target_categories = [x.split("_")[0] for x in target_names]
                    
                    random_distractors = np.random.choice(local_distractor_ids, split_h_trials * 2)
                    data["distractor_id"].extend([distractor_ids[x] for x in random_distractors])
                    data["distractor"].extend([distractors[x] for x in random_distractors])
                    data["target_name"].extend(np.repeat(target_names , split_h_trials))
                    data["target_cat"].extend(np.repeat(target_categories , split_h_trials))
                    
                    if cue != "Neutral":
                        data["expectation"].extend(["unexpected"] * split_h_trials * 2)
                    else:
                        data["expectation"].extend(["neutral"]* split_h_trials * 2)
                        
                    data["mask_ISI"].extend([mask] * split_h_trials * 2)
                    data["cue"].extend([cue] * split_h_trials * 2)
                    data["cue_color"].extend([cue_data["cue_color"][cue_id]]* split_h_trials * 2)
                    data["cue_letter"].extend([cue_data["cue_letters"][cue_id]]* split_h_trials * 2)
                    
        for i, h_cat in enumerate(high_cats):
            h_cat_stim = stimuli[np.char.count(stimuli, h_cat) > 0]
            targets = h_cat_stim[np.char.count(h_cat_stim, "mask") == 0]
            target_ids = np.arange(len(stimuli))[np.isin(stimuli, targets)]

            trial_collectos = []
            for mask in mask_type:
                h_trials = cue_data["high_prob_trials"][cue_id][i]
                split_h_trials = int(h_trials // 2)
                
                data["target_id"].extend(np.repeat(target_ids , split_h_trials))
                data["target"].extend(np.repeat(targets , split_h_trials))
                
                target_names =[x.split("\\")[-1] for x in targets]
                target_categories = [x.split("_")[0] for x in target_names]
                
                random_distractors = np.random.choice(local_distractor_ids, split_h_trials * 2)
                data["distractor_id"].extend([distractor_ids[x] for x in random_distractors])
                data["distractor"].extend([distractors[x] for x in random_distractors])
                data["target_name"].extend(np.repeat(target_names , split_h_trials))
                data["target_cat"].extend(np.repeat(target_categories , split_h_trials))
                
                
                if cue != "Neutral":
                    data["expectation"].extend(["expected"] * split_h_trials * 2)
                else:
                    data["expectation"].extend(["neutral"]* split_h_trials * 2)
                    
                data["mask_ISI"].extend([mask] * split_h_trials * 2)
                data["cue"].extend([cue] * split_h_trials * 2)
                data["cue_color"].extend([cue_data["cue_color"][cue_id]]* split_h_trials * 2)
                data["cue_letter"].extend([cue_data["cue_letters"][cue_id]] * split_h_trials * 2)
                

                
    data["target_loc"] = [np.random.choice(["L", "R"], 1, p=[0.5, 0.5])[0] for _ in range(len(data["target"]))]
    data["distractor_loc"] = ["R" if x == "L" else "L" for x in data["target_loc"]]
    df = pd.DataFrame(data)
    df = build_constrained_order(df, seed=random_seed)
    return df, stimuli
        

In [29]:
cue_data = create_cue_dynam()
stim_path = "/projects/crunchie/boyanova/EEG_Things/Mask_ExpAtt_EEG/stimuli"
trials, _ = create_block_trials(stim_path, cue_data, random_seed=19)

In [30]:
trials.head()

,target_id,distractor_id,target,distractor,expectation,mask_ISI,cue,cue_color,cue_letter,target_name,target_cat,target_loc,distractor_loc
0,11,5,.\stimuli\submarine\submarine_01.jpg,.\stimuli\speedboat\mask-speedboat_01.png,expected,0.017,Water vessel,"[-0.4, -0.2, 0.4]",WV,submarine_01.jpg,submarine,L,R
1,14,8,.\stimuli\whale\whale_00.jpg,.\stimuli\submarine\mask-submarine_00.png,unexpected,0.017,Water vessel,"[-0.4, -0.2, 0.4]",WV,whale_00.jpg,whale,R,L
2,6,3,.\stimuli\speedboat\speedboat_00.jpg,.\stimuli\dolphin\mask-dolphin_01.png,expected,0.100,Water vessel,"[-0.4, -0.2, 0.4]",WV,speedboat_00.jpg,speedboat,L,R
3,10,9,.\stimuli\submarine\submarine_00.jpg,.\stimuli\submarine\mask-submarine_01.png,neutral,0.017,Neutral,"[-1, -1, -1]",X,submarine_00.jpg,submarine,R,L
4,1,9,.\stimuli\dolphin\dolphin_01.jpg,.\stimuli\submarine\mask-submarine_01.png,unexpected,0.017,Water vessel,"[-0.4, -0.2, 0.4]",WV,dolphin_01.jpg,dolphin,R,L


In [1]:
import os
from PIL import Image

root_dir = "/projects/crunchie/boyanova/EEG_Things/Mask_ExpAtt_EEG/target_stimuli"
target_size = (500, 500)

# Image extensions to process
valid_exts = (".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp")

for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.lower().endswith(valid_exts):
            img_path = os.path.join(root, file)

            try:
                with Image.open(img_path) as img:
                    img = img.convert("RGB")  # safe for consistency
                    img_resized = img.resize(target_size, Image.LANCZOS)
                    img_resized.save(img_path)

            except Exception as e:
                print(f"Failed to process {img_path}: {e}")
